In [1]:
%pip install numpy
%pip install mediapipe
%pip install opencv-python
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import os
import mediapipe as mp
import csv
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from itertools import combinations

In [6]:
def joint_angles_fast(coords, combos):
    vecA = np.subtract(coords[combos[:, 1], :], coords[combos[:, 0], :])
    vecB = np.subtract(coords[combos[:, 1], :], coords[combos[:, 2], :])
    X = np.einsum('ij, ij -> i', vecA, vecB)/np.linalg.norm(vecA, 2, 1)/np.linalg.norm(vecB, 2, 1)
    Y = np.linalg.norm(np.cross(vecA, vecB), 2, 1)/np.linalg.norm(vecA, 2, 1)/np.linalg.norm(vecB, 2, 1)
    angles = np.arctan2(Y, X)
    return angles

# Set up combination vector so all the angles can be computed
landmarks = np.arange(21)
combos = np.array(list(combinations(landmarks, 3)))

# Set up image and label directories for iteration later
image_directory = 'C:/Users/erice/OneDrive/Desktop/MediaPipe-Hands/asl_alphabet_train/asl_alphabet_train'
#label_directory = os.path.join('train'+'\labels')

# STEP 2: Create an HandLandmarker object.
base_options = python.BaseOptions(model_asset_path='hand_landmarker.task')
options = vision.HandLandmarkerOptions(base_options=base_options,
                                       num_hands=2,
                                       running_mode= mp.tasks.vision.RunningMode.IMAGE)
detector = vision.HandLandmarker.create_from_options(options)
angles_array = []
label_array = []
for folder in os.scandir(image_directory):
    for file in os.scandir(folder):
        image = mp.Image.create_from_file(file.path)
        results = detector.detect(image)
        landmarks = []
        if results.hand_landmarks:
            for landmark in results.hand_landmarks[0]:
                coord = np.array([landmark.x, landmark.y, landmark.z])
                landmarks.append(coord)
            angles = joint_angles_fast(np.array(landmarks), combos)
            label = os.path.basename(folder)
            angles_array.append(angles)
            label_array.append(label)

c:\Python39\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
angle_header = []
for i in range(1330):
    angle_header.append("Angle " + str(i + 1))
df_angles = pd.DataFrame(angles_array, columns=angle_header)

In [8]:
df_angles.to_csv('angles_array.csv')

In [4]:
df = pd.read_csv('angles_array.csv', index_col = 0)

In [9]:
df_labels = pd.DataFrame(label_array, columns=["Label"])
df_labels.to_csv('label_array.csv')

In [10]:
df = pd.read_csv('label_array.csv', index_col = 0)